In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3651, -0.2540, -0.2620, -0.2995, -0.2839, -0.2131, -0.1594,  0.2123,
         -0.6255,  0.4270, -0.1250,  0.4684, -0.2043, -0.2625,  0.2994, -0.5509,
         -0.2679,  1.0322, -0.4132,  0.0818],
        [-0.2695, -0.1035,  0.1144, -0.5356, -0.2908, -0.4378, -0.2945,  0.1295,
         -0.6574,  0.0664,  0.1831,  0.1038, -0.2259, -0.2477,  0.4174, -0.2943,
          0.0444,  0.5896, -0.4636, -0.1801],
        [-0.4662, -0.0619, -0.0926, -0.0606, -0.3334, -0.4083, -0.3283, -0.0088,
         -0.6215,  0.3573,  0.0590,  0.0473, -0.2492, -0.4807,  0.1544, -0.4915,
          0.0055,  0.3386, -0.0175,  0.0018]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2123, 0.0000,
         0.4270, 0.0000, 0.4684, 0.0000, 0.0000, 0.2994, 0.0000, 0.0000, 1.0322,
         0.0000, 0.0818],
        [0.0000, 0.0000, 0.1144, 0.0000, 0.0000, 0.0000, 0.0000, 0.1295, 0.0000,
         0.0664, 0.1831, 0.1038, 0.0000, 0.0000, 0.41

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0087,  0.0259, -0.0335,  ...,  0.0009, -0.0237, -0.0015],
        [-0.0163,  0.0293, -0.0227,  ..., -0.0210,  0.0178, -0.0186]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0159, -0.0259], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0221, -0.0296, -0.0246,  ..., -0.0214,  0.0184, -0.0122],
        [-0.0409,  0.0363,  0.0289,  ..., -0.0029,  0.0135, -0.0202]],
       device='cuda:0', grad_fn=<Sl